# Model training and prediction - `s30d`

In [1]:
import os
os.chdir("..")

In [2]:
os.getcwd()

'/Users/ludvigwarnberggerdin/projects/python/pemett'

In [3]:
import numpy as np
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier, RandomForestClassifier

In [4]:
X_train = pd.read_csv("./data/processed/s30d/X_train.csv", index_col = 0)
y_train = pd.read_csv("./data/processed/s30d/y_train.csv", index_col = 0).s30d
X_test = pd.read_csv("./data/processed/s30d/X_test.csv", index_col = 0)
y_test = pd.read_csv("./data/processed/s30d/y_test.csv", index_col = 0).s30d

In [5]:
y_train.value_counts() / len(y_train.index) * 100

0.0    94.197074
1.0     5.802926
Name: s30d, dtype: float64

In [6]:
cont_features = ["age", "hr", "sbp", "dbp", "spo2", "rr", "delay"]
cat_features = list(X_train.loc[:, ~X_train.columns.isin(cont_features)].columns)

### Lightgbm

In [92]:
from lightgbm import LGBMClassifier

In [8]:
continous_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('cont', continous_transformer, cont_features)]
)

In [9]:
ss = StandardScaler()
X_train.loc[:, cont_features] = ss.fit_transform(X_train.loc[:, cont_features])
X_test.loc[:, cont_features] = ss.fit_transform(X_test.loc[:, cont_features])

In [10]:
clf = LGBMClassifier()

In [11]:
clf.fit(
    X = X_train,
    y = y_train,
    categorical_feature = cat_features
)

/Users/ludvigwarnberggerdin/miniforge3/envs/pemett/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier()

In [12]:
y_pred_prob_train = clf.predict_proba(X = X_train)
y_pred_prob_test = clf.predict_proba(X = X_test)
y_pred_train = clf.predict(X = X_train)
y_pred_test = clf.predict(X = X_test)

Report for continous scores

In [13]:
print(classification_report(y_true = y_train, y_pred = y_pred_train))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5860
         1.0       1.00      0.99      1.00       361

    accuracy                           1.00      6221
   macro avg       1.00      1.00      1.00      6221
weighted avg       1.00      1.00      1.00      6221



In [14]:
print(classification_report(y_true = y_test, y_pred = y_pred_test))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1954
         1.0       0.77      0.62      0.69       120

    accuracy                           0.97      2074
   macro avg       0.87      0.80      0.83      2074
weighted avg       0.96      0.97      0.97      2074



Gridsearch breaks for the continous score (to enable comparison with clinicians)

In [218]:
import copy
import random
from src.models.train_model import generate_all_combinations
from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold

def gridsearch_breaks(clf, hyper_parameters, all_breaks,
                      X_train, y_pred_prob_train, y_train, 
                      sample_size = None) -> tuple([tuple, list, pd.DataFrame]):
    """Gridsearch breaks for continous probabilites.
    
    Breaks are chosen based on validation set performance.
    """
    ## Sample combinations
    if sample_size:
        all_breaks = random.sample(all_breaks, round(sample_size * len(all_breaks)))
        
    ## Merge breaks and model hyper parameters
    d = {**hyper_parameters, 'breaks': all_breaks}
    hyper_parameters = generate_all_combinations(d)
    
    ## Compute predictions and perofmrance of each combination over five folds
    n_folds = 2
    preds = np.zeros((len(X_train.index), ))
    roc_aucs = pd.DataFrame(
        data = np.zeros((len(hyper_parameters), n_folds)),
        columns = range(1, n_folds + 1),
        index = [str(hp) for hp in hyper_parameters]
    )

    for i, (train_index, val_index) in enumerate(StratifiedKFold(n_splits = n_folds).split(X_train, y_train)):

        X_train_ = X_train.iloc[train_index]
        y_train_ = y_train.iloc[train_index]

        X_val = X_train.iloc[val_index]
        y_val = y_train.iloc[val_index]

        for j, hp in enumerate(tqdm(hyper_parameters)):

            hp_ = copy.deepcopy(hp)
            breaks = hp_.pop("breaks")

            clf = clf.set_params(**hp_)
            clf.fit(X_train_, y_train_)

            y_pred_val = clf.predict(X_val)
            y_pred_prob_val = clf.predict_proba(X_val)
            preds[val_index] = y_pred_prob_val[:, 1]

            binned_predictions = pd.cut(y_pred_prob_val[:, 1], breaks, labels = [0, 1, 2, 3], right = True, include_lowest = False)
            roc_aucs.iloc[j, i] = roc_auc_score(y_true = y_val, y_score = binned_predictions)

    return roc_aucs.mean(axis = 1), preds

In [219]:
import itertools as it
all_breaks = [(0, ) + x + (np.inf,) for x in it.combinations(np.arange(0.01, 1, 0.01), r=3)]

In [220]:
hyper_parameters = {'max_depth': [5]}

In [221]:
roc_aucs, preds = gridsearch_breaks(
    clf = LGBMClassifier(),
    hyper_parameters = hyper_parameters,
    all_breaks = all_breaks,
    X_train = X_train,
    y_pred_prob_train = y_pred_prob_train[:, 1], ## Predicted probabilities of 1s, i.e. dead within 30 days
    y_train = y_train, 
    sample_size = 0.0001
)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [264]:
import numpy as np
from scipy import sparse
from sklearn.base import ClassifierMixin


class _BaseStackingClassifier(ClassifierMixin):
    """Base class of stacking classifiers
    """

    def _do_predict(self, X, predict_fn):
        meta_features = self.predict_meta_features(X)

        if not self.use_features_in_secondary:
            return predict_fn(meta_features)
        elif sparse.issparse(X):
            return predict_fn(sparse.hstack((X, meta_features)))
        else:
            return predict_fn(np.hstack((X, meta_features)))

    def predict(self, X):
        """ Predict target values for X.

        Parameters
        ----------
        X : numpy array, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        labels : array-like, shape = [n_samples]
            Predicted class labels.

        """
        return self._do_predict(X, self.meta_clf_.predict)

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        proba : array-like, shape = [n_samples, n_classes] or a list of \
                n_outputs of such arrays if n_outputs > 1.
            Probability for each class per sample.

        """
        check_is_fitted(self, ['clfs_', 'meta_clf_'])

        return self._do_predict(X, self.meta_clf_.predict_proba)

    def decision_function(self, X):
        """ Predict class confidence scores for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        scores : shape=(n_samples,) if n_classes == 2 else \
            (n_samples, n_classes).
            Confidence scores per (sample, class) combination. In the binary
            case, confidence score for self.classes_[1] where >0 means this
            class would be predicted.

        """
        check_is_fitted(self, ['clfs_', 'meta_clf_'])

        return self._do_predict(X, self.meta_clf_.decision_function)


In [ ]:
def StackingClassifier(_BaseStackingClassifier):
    
    def __init__(self, )

In [255]:
def ensemble_learning(base_clfs, meta_clf, y_train, **kwargs):
    """Conduct ensemble learning.
    
    Inspired by:
        https://github.com/rasbt/mlxtend/blob/master/mlxtend/classifier/stacking_cv_classification.py
    """
    ## Fit each classifier to the original training set
    predictions = np.array([])
    clf_keys = base_clfs.keys()
    ## Get predicted probabilities on the training set for each classifier
    for clfk in clf_keys:
        d = clfs[clfk]
        clf = d['clf']
        hp = d['hp']
        print("Running predictions for " + str(clf))
        roc_aucs, preds = gridsearch_breaks(
            clf = clf,
            hyper_parameters = hp,
            y_train=y_train,
            **kwargs
        )
        if predictions.any():
            predictions = np.hstack([predictions, preds[:, np.newaxis]])
        else:
            predictions = preds[:, np.newaxis]

    ## Fit each classifier to the predicted probabilities
    for clfk in clf_keys:
        d = base_clfs[clfk]
        clf = d['clf']
        hp = d['hp']
        
        clf.fit(
            X=predictions, 
            y=y_train
        )
    ## Fit the meta classifier
    meta_clf.fit(predictions, y_train)
    
    return base_clfs, meta_clf

In [256]:
from sklearn.ensemble import RandomForestClassifier

In [257]:
lgbm = dict(
    clf=LGBMClassifier(),
    hp=dict(
        max_depth=[5]
    )
)
lr = dict(
    clf=LGBMClassifier(),
    hp=dict(
        max_depth=[100, 200]
    )
)
clfs = dict(
    lgbm=lgbm,
    lr=lr
)

In [261]:
clfs, meta_clf = ensemble_learning(
    base_clfs=clfs,
    meta_clf=LGBMClassifier(),
    all_breaks=all_breaks,
    X_train=X_train, 
    y_pred_prob_train=y_pred_prob_train,
    y_train = y_train,
    sample_size=0.0001
)

Running predictions for LGBMClassifier(max_depth=5)


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Running predictions for LGBMClassifier(max_depth=200)


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [262]:
meta_clf.predict(X_test)

ValueError: Number of features of the model must match the input. Model n_features_ is 2 and input n_features is 15